In [80]:
import pandas as pd 
import pm4py
import seaborn as sns
import matplotlib.pyplot as plt

In [81]:
log = pm4py.read_xes("../data/InternationalDeclarations.xes.gz")
df = pm4py.convert_to_dataframe(log)

parsing log, completed traces :: 100%|██████████| 6449/6449 [00:05<00:00, 1285.15it/s]


**Load the data in pandas DataFrame**

In [82]:
log_time = pm4py.read_xes("../data/International_Declarations_final.xes")
df_time = pm4py.convert_to_dataframe(log_time)  
log_time = pm4py.read_xes('../data/International_Declarations_final.xes')

parsing log, completed traces :: 100%|██████████| 5646/5646 [00:05<00:00, 1084.59it/s]


In [83]:
working_df = df_time
print(len(working_df.index))
working_df.head()

65376


,org:resource,org:role,remaining_time,id,concept:name,time:timestamp,case:Permit travel permit number,case:DeclarationNumber,case:Amount,case:RequestedAmount,...,case:id,case:Permit ID,case:Permit id,case:BudgetNumber,case:Permit ActivityNumber,case:AdjustedAmount,case:concept:name,trip_duration,remaining_time_since_end_trip,time_submission
0,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_6,Start trip,2016-10-05 00:00:00+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
1,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_7,End trip,2016-10-05 00:00:00+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,0.0,4577.514722,NaN
2,STAFF MEMBER,EMPLOYEE,171.978611,st_step 76459_0,Permit SUBMITTED by EMPLOYEE,2017-04-06 13:32:10+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
3,STAFF MEMBER,SUPERVISOR,171.973611,st_step 76460_0,Permit FINAL_APPROVED by SUPERVISOR,2017-04-06 13:32:28+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
4,STAFF MEMBER,EMPLOYEE,147.877500,st_step 76461_0,Declaration SUBMITTED by EMPLOYEE,2017-04-07 13:38:14+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN


In [84]:
eventColumnName = "concept:name"
caseIdColumnName = "case:id"
caseIds = working_df[caseIdColumnName].unique()

index_before_End = []

for caseId in caseIds:
    cur_trace = working_df[working_df[caseIdColumnName] == caseId]
    end_trip_time = pd.Timestamp(cur_trace[cur_trace[eventColumnName ] == "End trip"]["time:timestamp"].values[0])
    
    for i, event in cur_trace.iterrows():
        if pd.Timestamp(event["time:timestamp"]).replace(tzinfo=None) <= end_trip_time.tz_localize(None):
            index_before_End.append(i)
        
df_event_before_end = working_df.iloc[index_before_End].copy()
df_event_before_end

,org:resource,org:role,remaining_time,id,concept:name,time:timestamp,case:Permit travel permit number,case:DeclarationNumber,case:Amount,case:RequestedAmount,...,case:id,case:Permit ID,case:Permit id,case:BudgetNumber,case:Permit ActivityNumber,case:AdjustedAmount,case:concept:name,trip_duration,remaining_time_since_end_trip,time_submission
0,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_6,Start trip,2016-10-05 00:00:00+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
1,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_7,End trip,2016-10-05 00:00:00+02:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,0.0,4577.514722,NaN
8,STAFF MEMBER,EMPLOYEE,2705.517222,rv_travel permit 76665_6,Start trip,2016-11-21 00:00:00+01:00,travel permit number 76666,declaration number 76668,346.544903,346.544903,...,declaration 76667,travel permit 76665,travel permit 76665,budget 144054,UNKNOWN,346.544903,declaration 76667,NaN,NaN,NaN
9,STAFF MEMBER,EMPLOYEE,1961.517222,rv_travel permit 76665_7,End trip,2016-12-22 00:00:00+01:00,travel permit number 76666,declaration number 76668,346.544903,346.544903,...,declaration 76667,travel permit 76665,travel permit 76665,budget 144054,UNKNOWN,346.544903,declaration 76667,744.0,1961.517222,NaN
16,STAFF MEMBER,EMPLOYEE,1025.530556,rv_travel permit 73652_6,Start trip,2016-12-08 00:00:00+01:00,travel permit number 73653,declaration number 73655,56.972769,56.972769,...,declaration 73654,travel permit 73652,travel permit 73652,budget 143677,UNKNOWN,56.972769,declaration 73654,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65365,STAFF MEMBER,ADMINISTRATION,1082.501389,st_step 13460_0,Permit APPROVED by ADMINISTRATION,2018-12-10 15:01:39+01:00,travel permit number 13457,declaration number 13459,445.064943,445.064943,...,declaration 13458,travel permit 13456,travel permit 13456,budget 147525,UNKNOWN,445.064943,declaration 13458,NaN,NaN,NaN
65366,STAFF MEMBER,BUDGET OWNER,1064.014444,st_step 13461_0,Permit APPROVED by BUDGET OWNER,2018-12-11 09:30:52+01:00,travel permit number 13457,declaration number 13459,445.064943,445.064943,...,declaration 13458,travel permit 13456,travel permit 13456,budget 147525,UNKNOWN,445.064943,declaration 13458,NaN,NaN,NaN
65367,STAFF MEMBER,SUPERVISOR,900.273056,st_step 13463_0,Permit FINAL_APPROVED by SUPERVISOR,2018-12-18 05:15:21+01:00,travel permit number 13457,declaration number 13459,445.064943,445.064943,...,declaration 13458,travel permit 13456,travel permit 13456,budget 147525,UNKNOWN,445.064943,declaration 13458,NaN,NaN,NaN
65368,STAFF MEMBER,EMPLOYEE,857.528889,rv_travel permit 13456_6,Start trip,2018-12-20 00:00:00+01:00,travel permit number 13457,declaration number 13459,445.064943,445.064943,...,declaration 13458,travel permit 13456,travel permit 13456,budget 147525,UNKNOWN,445.064943,declaration 13458,NaN,NaN,224.974444


In [85]:
df_event_before_end.columns

Index(['org:resource', 'org:role', 'remaining_time', 'id', 'concept:name',
       'time:timestamp', 'case:Permit travel permit number',
       'case:DeclarationNumber', 'case:Amount', 'case:RequestedAmount',
       'case:Permit TaskNumber', 'case:Permit BudgetNumber',
       'case:OriginalAmount', 'case:Permit ProjectNumber',
       'case:Permit OrganizationalEntity', 'case:travel permit number',
       'case:Permit RequestedBudget', 'case:id', 'case:Permit ID',
       'case:Permit id', 'case:BudgetNumber', 'case:Permit ActivityNumber',
       'case:AdjustedAmount', 'case:concept:name', 'trip_duration',
       'remaining_time_since_end_trip', 'time_submission'],
      dtype='object')

In [86]:
eventCategories = df_event_before_end[eventColumnName].unique()
caseIds = df_event_before_end[caseIdColumnName].unique()

frequency_df = pd.DataFrame(index=caseIds, columns=eventCategories).fillna(0)

for cur_case in caseIds:
    cur_eventCategories = df_event_before_end[df_event_before_end[caseIdColumnName] == cur_case][eventColumnName]
    for e in cur_eventCategories:
        frequency_df.loc[cur_case][e] += 1  

In [87]:
frequency_df

,Start trip,End trip,Permit SUBMITTED by EMPLOYEE,Permit APPROVED by SUPERVISOR,Permit FINAL_APPROVED by DIRECTOR,Permit FINAL_APPROVED by SUPERVISOR,Permit APPROVED by PRE_APPROVER,Permit REJECTED by SUPERVISOR,Permit REJECTED by EMPLOYEE,Permit REJECTED by MISSING,...,Declaration REJECTED by SUPERVISOR,Permit APPROVED by ADMINISTRATION,Permit APPROVED by BUDGET OWNER,Declaration APPROVED by ADMINISTRATION,Declaration APPROVED by BUDGET OWNER,Declaration REJECTED by ADMINISTRATION,Declaration REJECTED by EMPLOYEE,Permit REJECTED by ADMINISTRATION,Permit REJECTED by BUDGET OWNER,Declaration REJECTED by BUDGET OWNER
declaration 76457,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
declaration 76667,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
declaration 73654,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
declaration 73582,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
declaration 72590,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
declaration 12674,1,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
declaration 15973,1,1,1,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
declaration 12689,1,1,1,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
declaration 15436,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
df_event_before_end["time_submission"] = df_event_before_end.groupby("case:concept:name")["time_submission"].transform(lambda x: x.fillna(x.mean()))
df_event_before_end["time_submission"] = df_event_before_end.groupby("case:concept:name")["time_submission"].transform(lambda x: x.fillna(0))
df_event_before_end["trip_duration"] = df_event_before_end.groupby("case:concept:name")["trip_duration"].transform(lambda x: x.fillna(x.mean()))
df_event_before_end["remaining_time_since_end_trip"] = df_event_before_end.groupby("case:concept:name")["remaining_time_since_end_trip"].transform(lambda x: x.fillna(x.mean()))

In [89]:
df_event_before_end.loc[df_event_before_end["case:concept:name"] == "declaration 13458"]

,org:resource,org:role,remaining_time,id,concept:name,time:timestamp,case:Permit travel permit number,case:DeclarationNumber,case:Amount,case:RequestedAmount,...,case:id,case:Permit ID,case:Permit id,case:BudgetNumber,case:Permit ActivityNumber,case:AdjustedAmount,case:concept:name,trip_duration,remaining_time_since_end_trip,time_submission
65364,STAFF MEMBER,EMPLOYEE,1082.503333,st_step 13462_0,Permit SUBMITTED by EMPLOYEE,2018-12-10 15:01:32+01:00,travel permit number 13457,declaration number 13459,445.064943,445.064943,...,declaration 13458,travel permit 13456,travel permit 13456,budget 147525,UNKNOWN,445.064943,declaration 13458,0.0,857.528889,224.974444
65365,STAFF MEMBER,ADMINISTRATION,1082.501389,st_step 13460_0,Permit APPROVED by ADMINISTRATION,2018-12-10 15:01:39+01:00,travel permit number 13457,declaration number 13459,445.064943,445.064943,...,declaration 13458,travel permit 13456,travel permit 13456,budget 147525,UNKNOWN,445.064943,declaration 13458,0.0,857.528889,224.974444
65366,STAFF MEMBER,BUDGET OWNER,1064.014444,st_step 13461_0,Permit APPROVED by BUDGET OWNER,2018-12-11 09:30:52+01:00,travel permit number 13457,declaration number 13459,445.064943,445.064943,...,declaration 13458,travel permit 13456,travel permit 13456,budget 147525,UNKNOWN,445.064943,declaration 13458,0.0,857.528889,224.974444
65367,STAFF MEMBER,SUPERVISOR,900.273056,st_step 13463_0,Permit FINAL_APPROVED by SUPERVISOR,2018-12-18 05:15:21+01:00,travel permit number 13457,declaration number 13459,445.064943,445.064943,...,declaration 13458,travel permit 13456,travel permit 13456,budget 147525,UNKNOWN,445.064943,declaration 13458,0.0,857.528889,224.974444
65368,STAFF MEMBER,EMPLOYEE,857.528889,rv_travel permit 13456_6,Start trip,2018-12-20 00:00:00+01:00,travel permit number 13457,declaration number 13459,445.064943,445.064943,...,declaration 13458,travel permit 13456,travel permit 13456,budget 147525,UNKNOWN,445.064943,declaration 13458,0.0,857.528889,224.974444
65369,STAFF MEMBER,EMPLOYEE,857.528889,rv_travel permit 13456_7,End trip,2018-12-20 00:00:00+01:00,travel permit number 13457,declaration number 13459,445.064943,445.064943,...,declaration 13458,travel permit 13456,travel permit 13456,budget 147525,UNKNOWN,445.064943,declaration 13458,0.0,857.528889,224.974444


In [90]:
time_sub = []
trip_duration = []
remaining_time = []
start_time_event = []
amount = []


for caseId in caseIds:
    cur_trace = df_event_before_end.loc[df_event_before_end[caseIdColumnName] == caseId]
    time_sub.append(cur_trace["time_submission"].iloc[0])
    trip_duration.append(cur_trace["trip_duration"].iloc[0])
    remaining_time.append(cur_trace["remaining_time_since_end_trip"].iloc[0])
    start_time_event.append(pd.Timestamp(cur_trace["time:timestamp"].iloc[0]))
    amount.append(cur_trace["case:Amount"].iloc[0])

In [91]:
frequency_df["start_time_trace"] = start_time_event
frequency_df["remaining_time_since_end_trip"] = remaining_time
frequency_df["trip_duration"] = trip_duration
frequency_df["time_submission"] = time_sub
frequency_df["amount"] = amount


In [92]:
frequency_df.sort_values(by='start_time_trace')

,Start trip,End trip,Permit SUBMITTED by EMPLOYEE,Permit APPROVED by SUPERVISOR,Permit FINAL_APPROVED by DIRECTOR,Permit FINAL_APPROVED by SUPERVISOR,Permit APPROVED by PRE_APPROVER,Permit REJECTED by SUPERVISOR,Permit REJECTED by EMPLOYEE,Permit REJECTED by MISSING,...,Declaration REJECTED by ADMINISTRATION,Declaration REJECTED by EMPLOYEE,Permit REJECTED by ADMINISTRATION,Permit REJECTED by BUDGET OWNER,Declaration REJECTED by BUDGET OWNER,start_time_trace,remaining_time_since_end_trip,trip_duration,time_submission,amount
declaration 76457,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2016-10-05 00:00:00+02:00,4577.514722,0.0,0.000000,39.664561
declaration 76667,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2016-11-21 00:00:00+01:00,1961.517222,744.0,0.000000,346.544903
declaration 73654,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2016-12-08 00:00:00+01:00,1025.530556,0.0,0.000000,56.972769
declaration 73582,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,2017-01-01 00:00:00+01:00,617.518889,8736.0,0.000000,199.546260
declaration 72590,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2017-01-09 00:00:00+01:00,809.538333,48.0,0.000000,14.569044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
declaration 12674,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,2018-12-07 13:00:16+01:00,1361.522500,24.0,34.995556,43.782713
declaration 15973,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,2018-12-07 22:32:45+01:00,1001.528889,48.0,97.454167,925.091226
declaration 12689,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,2018-12-09 17:41:05+01:00,419.987778,24.0,150.315278,297.421020
declaration 15436,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2018-12-10 00:00:00+01:00,809.522222,48.0,0.000000,317.596586


In [93]:
n = len(start_time_event)
train_set_size = int(n*0.8)

x_train = frequency_df.iloc[:train_set_size, :].drop("remaining_time_since_end_trip", axis=1)
y_train = remaining_time[:train_set_size]
y_train = frequency_df.iloc[train_set_size:, :].drop("remaining_time_since_end_trip", axis=1)
y_test = remaining_time[train_set_size:]

In [4]:
# df_time["trip_duration"] = df_time.groupby("case:concept:name")["trip_duration"].transform(lambda x: x.fillna(x.mean()))
# df_time["remaining_time_since_end_trip"] = df_time.groupby("case:concept:name")["remaining_time_since_end_trip"].transform(lambda x: x.fillna(x.mean()))
# df_time["time_submission"] = df_time.groupby("case:concept:name")["time_submission"].transform(lambda x: x.fillna(x.mean()))
# df_time["time_submission"] = df_time.groupby("case:concept:name")["time_submission"].transform(lambda x: x.fillna(0))
# df_time["decl_rejected"] = df_time.groupby("case:concept:name")["concept:name"].transform(lambda x: True if "Declaration REJECTED by ADMINISTRATION" in x.values else False)
# df_time["decl_rejected"] = df_time["decl_rejected"].replace({True: 1, False: 0})
# df_time["permit_budget_owner"] = df_time.groupby("case:concept:name")["concept:name"].transform(lambda x: True if "Permit APPROVED by BUDGET OWNER" in x.values else False)
# df_time["permit_budget_owner"] = df_time["permit_budget_owner"].replace({True: 1, False: 0})

In [5]:
print("Correlation with the presence of a budget owner in the process: " + str(df_time["remaining_time_since_end_trip"].corr(df_time["permit_budget_owner"])))
print("Correlation with the duration of the trip: " + str(df_time["remaining_time_since_end_trip"].corr(df_time["trip_duration"])))
print("Correlation with the fact that the declaration is Rejected or not: " + str(df_time["remaining_time_since_end_trip"].corr(df_time["decl_rejected"])))
print("Correlation with the submission time in the process: " + str(df_time["remaining_time_since_end_trip"].corr(df_time["time_submission"])))

Correlation with the presence of a budget owner in the process: 0.0181617125650545
Correlation with the duration of the trip: 0.047811694303603956
Correlation with the fact that the declaration is Rejected or not: 0.12215296566622141
Correlation with the submission time in the process: -0.15312712076685048
